In [2]:
import os
import csv
import adin.settings as st
from IPython.display import display
from references.forms.address_forms import AddressCreateModelForm
from django.contrib.auth.models import User
from os.path import join
from adin.settings import BASE_DIR

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [3]:
def data_load(load_info, load_list=None):
    timers = {}
    counter = 0
    if load_list:
        for element in load_list:
            model_load(load_info[element])
    else:
        for value in load_info.values():
            model_load(value)
    
def model_load(load_dict):
    df_from_csv = pd.read_csv(join(BASE_DIR, f"_files/exports/{load_dict['csv_name']}"), keep_default_na=False)\
        .drop(load_dict['to_drop'], axis=1)\
        .rename(columns=load_dict['to_rename'])
    data_dict = df_from_csv\
        .assign(**{column: df_from_csv[column].apply(lambda x: None if x == '' else x) for column in df_from_csv.columns})\
        .assign(**{column: df_from_csv[column].apply(lambda x: model.objects.get(pk=x)) for column, model in load_dict['fk_dict'].items()})\
        .assign(state=df_from_csv.state.apply(lambda x: 2))\
        .to_dict('index')
    load_dict['model'].objects.bulk_create([load_dict['model'](**data) for key, data in data_dict.items()])

In [4]:
from scripts.utils import m2m_info

load_dict = m2m_info['accountable_transaction_type']

df_from_csv = pd.read_csv(join(BASE_DIR, f"_files/{load_dict['csv_name']}"), keep_default_na=False)
df_from_csv

,accountable,accountable_transaction_type
0,C18-1N-41^2021-02-01,Arriendo Inmueble Comercial
1,K1-17-43^2017-07-15,Arriendo Inmueble Comercial
2,K1-17-37^2012-02-01,Arriendo Inmueble Comercial
3,C18-1N-39^2022-03-01,Arriendo Inmueble Comercial
4,C18-1N-33^2021-10-01,Arriendo Inmueble Comercial
...,...,...
68,A6N-13N-62-S308^2022-03-01,Arriendo Inmueble Residencial
69,A6N-13N-62-S309^2022-03-01,Arriendo Inmueble Residencial
70,A6N-13N-62-S311^2021-11-01,Arriendo Inmueble Residencial
71,A6N-13N-62-S312^2007-06-01,Arriendo Inmueble Residencial


In [5]:
non_field_objs = load_dict['non_field_obj_model'].objects.filter(pk__in=list(df_from_csv[load_dict['non_field_obj_column']].unique()))
non_field_objs

<QuerySet [<Transaction: Arriendo Inmueble Comercial>, <Transaction: Arriendo Inmueble Residencial>]>

In [6]:
objs_df = df_from_csv.assign(**{load_dict['field_obj_column']:df_from_csv[load_dict['field_obj_column']].apply(lambda x: load_dict['field_obj_model'].objects.get(pk=x))})
objs_df.iloc[0]['accountable']

<Accountable: C18-1N-41^2021-02-01>

In [7]:
objs_df.apply(lambda x: eval(f"x[load_dict['field_obj_column']].{load_dict['m2m_field']}.add(non_field_objs.get(pk=x[load_dict['non_field_obj_column']]))"), axis=1)

0     None
1     None
2     None
3     None
4     None
      ... 
68    None
69    None
70    None
71    None
72    None
Length: 73, dtype: object

In [22]:
df_from_csv.iloc[0][load_dict['field_obj_column']]

'C18-1N-41^2021-02-01'